In [1]:
import socket
import sys
import requests
import requests_oauthlib
import json
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

# variables that will be used in OAuth for connecting to Twitter
ACCESS_TOKEN = '916682753847857157-P1KOyCUYS2ksnpn0NxURNbix3mzU5tg'
ACCESS_SECRET = 'j8GlirPEbuSop7oVbfqHvevAqpTLpIByn8ezluc2n3sGK'
CONSUMER_KEY = 'XZ86Aa2BOfpNbiBer4Y1N1aCR'
CONSUMER_SECRET = 'I7wLRHoT6GwfeNIz5rI4tACYJpQtdW2LETAsgI9Vi1nWbaFEqR'

# create our own Listener Class inheriting StreamListener in tweepy
class listener(StreamListener):
    def on_data(self,data):
        print(data)
        return(True)
    def on_error(self, status):
        print(status)

# Get real-time tweets:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
twitterStream = Stream(auth, listener())
twitterStream.filter(track=["hadoop","hive","spark"])

{"created_at":"Tue Nov 26 08:10:07 +0000 2019","id":1199238891787759616,"id_str":"1199238891787759616","text":"RT @LabourAgainstAS: Labour spark anger by questioning independence of equalities watchdog investigating them over #LabourParty:\n- Party vo\u2026","source":"\u003ca href=\"https:\/\/mobile.twitter.com\" rel=\"nofollow\"\u003eTwitter Web App\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1961233440,"id_str":"1961233440","name":"Nuddering","screen_name":"NudderingNudnik","location":null,"url":null,"description":"Nuddering so you don't have to. Globally sinister. Used to be a bitter Blairite failed muso. Now the Ray Winstone of anti-antisemitism & Marlon\u2019s best friend.","translator_type":"none","protected":false,"verified":false,"followers_count":5377,"friends_count":1203,"listed_count":59,"favourites_count":28665,"statuses_c

KeyboardInterrupt: 

In [2]:
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
# call APL URL and return the response for a stream of tweets
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'),('track', '%23')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

# send every tweet to Spark Streaming instance through a TCP connection
def send_tweets_to_spark(http_resp, top_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            print("Tweet Text: "+ tweet_text)
            print("-----------------")
            top_connection.send((tweet_text + '\n').encode())
        except:
            e = sys.exc_info()[0]
            print("error: %s" % e)

In [ ]:
TCP_IP = 'localhost'
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#s.setsockopt(socket.SQL_SOCKET,socket.SO_REUSEADDR, 1)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP Connection....")
conn, addr = s.accept()
print("connected, start getting tweets...")
resp = get_tweets()
send_tweets_to_spark(resp,conn)

Waiting for TCP Connection....
connected, start getting tweets...
https://stream.twitter.com/1.1/statuses/filter.json?language=en&track=%23 <Response [200]>
Tweet Text: RT @M00NP3RI: —

                  ## 𝐂𝐚𝐦 #
         𝙨𝙩𝙪𝙙𝙚𝙣𝙩¡𝙖𝙪 ⛓️ 𝐝𝐚𝐧𝐜𝐞𝐫
                   𝐏𝐋 / 𝐄𝐍𝐆

         ♡∕↻– [##𝐅𝐎𝐋𝐋𝐎𝐖]
— https://t…
-----------------
Tweet Text: Do you want the ref #
-----------------
Tweet Text: Hey guys, check out  and get started on building your new website and reach beyond your local and seasonal business… https://t.co/Qtw9hs4oRP
-----------------
Tweet Text: RT @STUDIONNGxLOEY: Tomorrow don't forget to use #/HBDLOEY in the birthday messages for ChanYeol. Please use it tomorrow at the birthday ti…
-----------------
Tweet Text: RT @bty_backup: I vote Surbhi Chandna for 

Sexiest star of the Decade
#SurbhiChandna 
Sexiest star of the Year
#SurbhiChandna

Check the b…
-----------------
Tweet Text: RT @YOUNGFLASHE: NEW HITS
Click on The Link To Play This Song 🔗💣💰💧🔗💰💣💦🔗 #new #music #song #stud